In [3]:
# Neuman Formula

# For calculating the mutual inductance between 2 (soon 4) coils of wire
# Currently assumes N=1

import numpy as np
from scipy.integrate import cumtrapz, dblquad # scipy version = 1.7.3
import scipy

# Constants
mu_0 = 1.2566 * 10 ** (-6)        # Vacuum permeability in H/m

# coil constants (all in meters)
R_1 = 0.05  # ground coil radius
R_2 = 0.025 # drone coil radii
r = 0.03    # axis offset # TODO ned two offsets now
z = 0.05    # plane offset
s = 0.1     # distance between centers of adjacent drone coils (side length of a square)

# calculate M

def f(phi2, phi1): # this is the Neumann formula for two circles with radii R_1 and R_2 with parallel axes, offset by r and z
    return np.cos(phi1 - phi2) / np.sqrt( R_1**2 + R_2**2 + r**2 + z**2 - 2*R_1*R_2*np.cos(phi1 - phi2) - 2*r*(R_1*np.sin(phi1) - R_2*np.sin(phi2)) )

M, M_error = mu_0 / (4 * np.pi) * np.array(dblquad(f, 0, 2*np.pi, lambda x: 0, lambda x: 2*np.pi))

print(f"M = {M}, error = {M_error}")

M = 5.011637732794961e-06, error = 1.0947014552717775e-14


In [16]:
# Slobodan Babic et al formula, ripped from their MATLAB file Babic_24.m

import numpy as np
from scipy.special import ellipk, ellipe


# TODO - there is a divergent case mentioned in the paper - implement the fix (if it already in there)
def calculate_M(Rp, Rs, pc, n, tol=1e-13):
    '''
    Calcualte the mutual inductance M.

    Keyword arguments:
    Rp  -- radius of primary (transmitting) loop
    Rs  -- radius of secondary (receiving) loop, requires Rp >= Rs
    pc  -- numpy.array() vector [x, y, z] between centers of loops
    n   -- numpy.array() vector [x, y, z] normal to rx loop
    tol -- absolute tolerance, default 1e-13
    '''

    def f24(p, h, e, g, d, a, b, c):
        '''
        idk
        p - vector
        h - number
        e - number
        g - number
        d - number
        a - number
        b - number
        c - number
        '''

        h2, e2, g2, a2, b2, c2 = h*h, e*e, g*g, a*a, b*b, c*c
        l2 = a*a+c2
        l = np.sqrt(l2)
        L2 = l2+b*b
        L = np.sqrt(L2)
        l2L2 = L2*l2
        lL = l*L

        sp = np.sin(p)
        cp = np.cos(p)
        cp2 = cp*cp
        sp2 = sp*sp

        if l == 0:
            p1, p2, p3, p4, p5 = 0, -g*np.sign(b), 0, -e*np.sign(b), d
            V = np.sqrt(e2 + g2 + h2*cp2 - 2*h*e*np.sign(b)*cp)
        else:
            p1, p2, p3, p4, p5 = g*c/l, -(e*l2 + g*a*b)/lL, h*c/L, (g*l2 - e*a*b - d*b*c)/lL, (d*a - e*c)/l
            V = np.sqrt( (e2 + g2) + h2*( (1 - b2*c2/l2L2)*cp2 + c2/l2*sp2 + a*b*c/(l2*L)*np.sin(2*p) ) - 2*h/lL*(e*a*b - g*l2)*cp - 2*h*e*c/l*sp )

        A = (1 + e2 + g2 + h2 + d*d) + 2*h*(p4*cp + p5*sp)
        m = 4*V/(A + 2*V)
        k = np.sqrt(m)
        K, E = ellipk(m), ellipe(m)
        PSI = (1 - 0.5*m)*K - E
        f = (p1*cp + p2*sp + p3)*PSI/(k*V**1.5)
        return f


    xc, yc, zc = pc[0], pc[1], pc[2]
    a, b, c = n[0], n[1], n[2]

    alpha, beta, gamma, delta = Rs/Rp, xc/Rp, yc/Rp, zc/Rp

    decdigs = int(np.abs(np.floor(np.log10(tol))))
    rom = np.zeros((2, decdigs))
    romall = np.zeros((1, 2**(decdigs-1) + 1))
    step = 2*np.pi/2**(decdigs-1)
    romall = f24(np.arange(0, 2*np.pi + 0.1*step, step), alpha, beta, gamma, delta, a, b, c)
    h = 2 * np.pi
    rom[0, 0] = h*(romall[0] + romall[-1])/2

    for i in range(2, decdigs + 1):
        step = 2**(decdigs - i + 1)
        # trapezoidal approximations
        indicies = [int(ind) for ind in np.arange((step/2) + 1, 2**(decdigs-1) + 0.1*step, step) - 1]
        rom[1, 0] = (rom[0, 0] + h*np.sum(romall[indicies]))/2
        # Richardson extrapolation
        for k in range(1, i):
            rom[1, k] = ((4**k)*rom[1, k - 1] - rom[0, k - 1])/((4**k) - 1)

        rom[0, 0:i] = rom[1, 0:i]
        h = h / 2

    M = 4e-7 * Rs * rom[0, decdigs - 1]
    return M

# testing it with Example_10 params
Rp=0.16
Rs=0.10
x1=0.2
x2=0.05
theta=60*(np.pi/180)
h=x1-x2*np.cos(theta)
d=x2*np.sin(theta)
psi=0*(np.pi/180)

A=Rp
a=Rs
rho=d
d=h

Rp=A
Rs=a
xc=0
yc=rho
zc=d
a=np.sin(psi)*np.sin(theta)
b=-np.cos(psi)*np.sin(theta)
c=np.cos(theta)

M = calculate_M(Rp, Rs, np.array([xc, yc, zc]), np.array([a, b, c]))
print("Example 10")
print(f"M = {M*1e9} nH")
print(f"Result of Babic_24 = 13.6112 nH")

# testing it with Example_11 params
Rp=0.40
Rs=0.10
x1=0.10
x2=0.20
theta=90*(np.pi/180)
h=x1-x2*np.cos(theta)
d=x2*np.sin(theta)
psi=0*(np.pi/180)

A=Rp
a=Rs
rho=d
d=h

Rp=A
Rs=a
xc=0
yc=rho
zc=d
a=np.sin(psi)*np.sin(theta)
b=-np.cos(psi)*np.sin(theta)
c=np.cos(theta)

M = calculate_M(Rp, Rs, np.array([xc, yc, zc]), np.array([a, b, c]))
print("Example 11")
print(f"M = {M*1e9} nH")
print(f"Result of Babic_24 = -10.7272 nH")


Example 10
M = 13.611247921416625 nH
Result of Babic_24 = 13.6112 nH
Example 11
M = -10.727151678661135 nH
Result of Babic_24 = -10.7272 nH
